# <left style="font-size:1.0em;">**About this Code**</left>




<div style="text-align:justify; font-size:1.2em;">
    
This code is consisting of architecture of 9 Deep learning and 5 Machine learning algorithms. Models are trained firstly using Radar features in 30 days and 15 days time difference between the TanDEM-X and ICESat-2 data acquisition dates. Further models are trained using both Radar and Environmental features from ECMWF data for 30 days and 15 days time difference. The step of data selection is done in the data preprocessing part. After the training of model, it is applied to the validation data set. History of Deep learning algorithms can also be saved. Further models are evaluated on the test data which is included in the next step of model evaluation and bias estimation.   
</div> 



In [ ]:
from tensorflow.keras.layers import Input, Add, LeakyReLU, BatchNormalization, Conv1D, Flatten, MaxPooling1D, Reshape, MultiHeadAttention, LayerNormalization, Dense, Input, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import os
import json
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [ ]:
# Directory to save models and histories
output_dir ="/folder/Save_results/"
os.makedirs(output_dir, exist_ok=True)

# Function to save the model and its training history
def save_model_and_history(model, history, model_name):
    # Save the model
    model_path = os.path.join(output_dir, f"{model_name}.h5")
    model.save(model_path)
    print(f"Model saved to {model_path}")

    # Save the history
    history_path = os.path.join(output_dir, f"{model_name}_history.json")
    with open(history_path, "w") as f:
        json.dump(history.history, f)
    print(f"History saved to {history_path}")


### 1-Fully Connected NN

In [ ]:
print('Fully Connected')
model = Sequential()
model.add(Input(shape=X_train.shape[1],))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer=Adam(learning_rate=0.0008), loss='mean_squared_error')
# Train the model
history = model.fit(X_train, y_train_np, epochs=100, batch_size=64,  validation_data=(X_val, y_val_np), verbose=1)

### 2-Batch Normalization NN

In [ ]:
print('Fully Batch')
modelBatch = Sequential()
modelBatch.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
modelBatch.add(BatchNormalization())
modelBatch.add(Dense(64, activation='relu'))
modelBatch.add(BatchNormalization())
modelBatch.add(Dense(1, activation='linear'))
modelBatch.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')
# Train the model
historyBatch = modelBatch.fit(X_train, y_train_np, epochs=100, batch_size=64,  validation_data=(X_val, y_val_np), verbose=1)


### 3-Leaky ReLU NN

In [ ]:
model_leaky = Sequential()
model_leaky.add(Dense(128, input_dim=X_train.shape[1]))
model_leaky.add(LeakyReLU(alpha=0.1))
model_leaky.add(BatchNormalization()) 
model_leaky.add(Dense(64))
model_leaky.add(LeakyReLU(alpha=0.1))
model_leaky.add(BatchNormalization())
model_leaky.add(Dense(1, activation='linear'))
model_leaky.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')
# Train the model
historyLeaky = model_leaky.fit(X_train, y_train_np, epochs=100, batch_size=64,  validation_data=(X_val, y_val_np), verbose=1)

### 4-Residual NN

In [ ]:
input_layer = Input(shape=(X_train.shape[1],))
x = Dense(128, activation='relu')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = BatchNormalization()(x) 
residual = Dense(128, activation='relu')(input_layer)  # Residual connection
x = Add()([x, residual])
x = Dense(64, activation='relu')(x)  
x = Dense(32, activation='relu')(x)
output_layer = Dense(1, activation='linear')(x)
model_residual = Model(inputs=input_layer, outputs=output_layer)
model_residual.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')
# Train the model
history_residual = model_residual.fit(X_train, y_train_np, epochs=100, batch_size=64,  validation_data=(X_val, y_val_np), verbose=1)

### 5-DNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
print('Deep')

model_deep = Sequential([
    Dense(1024, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),  
    Dropout(0.2),
    
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.1),
    
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.1),
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(1, activation='linear')
])


optimizer = Adam(learning_rate=0.0005)
model_deep.compile(optimizer=optimizer, loss='mean_squared_error')


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1
)

# Train the model
history_deep = model_deep.fit(
    X_train, y_train_np, 
    epochs=100, 
    batch_size=256,  
    validation_data=(X_val, y_val_np), 
    callbacks=[early_stopping,lr_scheduler],  
    verbose=1
)

### 6-Transformer Multi-Head Attention NN

In [ ]:
# Define input
input_layer = Input(shape=(X_train.shape[1],))
# Add a Dense layer to expand dimensions for attention mechanism
x = Dense(64, activation='relu')(input_layer)

# Reshape to fit MultiHeadAttention
x = tf.expand_dims(x, axis=1)  # Add temporal dimension
# MultiHeadAttention layer
attention_output = MultiHeadAttention(num_heads=8, key_dim=32)(x, x)
# Add normalization
x = LayerNormalization(epsilon=1e-6)(attention_output + x)
# Flatten and add dense layers
ffn = Dense(128, activation='relu')(x)
ffn = Dense(64, activation='relu')(ffn)
x = LayerNormalization(epsilon=1e-6)(x + ffn) 
x = tf.squeeze(x, axis=1)  # Remove the temporal dimension
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output_layer = Dense(1, activation='linear')(x)
# Build and compile the model
model_attention = Model(inputs=input_layer, outputs=output_layer)
model_attention.compile(optimizer=Adam(learning_rate=0.0005), loss='mean_squared_error')
# Train the model
history_attention = model_attention.fit(X_train, y_train_np, epochs=100, batch_size=64, validation_data=(X_val, y_val_np), verbose=1)

### 7-Auto Encoder NN

In [ ]:
# Define the autoencoder
input_layer = Input(shape=(X_train.shape[1],))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(X_train.shape[1], activation='linear')(decoded)
autoencoder = Model(inputs=input_layer, outputs=decoded)

# Encoder for regression task
encoder = Model(inputs=input_layer, outputs=encoded)
# Regression model
encoded_input = Input(shape=(32,))
regression = Dense(32, activation='relu')(encoded_input) 

regression_output = Dense(1, activation='linear')(regression)
regression_model = Model(inputs=encoded_input, outputs=regression_output)
# Compile and train the autoencoder
autoencoder.compile(optimizer=Adam(learning_rate=0.008), loss='mse')

autoencoder.fit(X_train, X_train, epochs=100, batch_size=64, validation_data=(X_val, X_val), shuffle=True, verbose=1)
# Use encoded features for regression
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)
X_val_encoded = encoder.predict(X_val)
regression_model.compile(optimizer=Adam(learning_rate=0.008), loss='mse')
#Train the model
history_encoder = regression_model.fit(
    X_train_encoded, y_train_np, epochs=100, batch_size=64, 
    validation_data=(X_val_encoded, y_val_np), shuffle=True, verbose=1
)

### 8-MLP NN

In [ ]:
# Define the MLP architecture
def create_mlp(input_dim, output_dim, task_type="regression"):
    """
    Create a Multi-Layer Perceptron (MLP) model.

    Parameters:
    - input_dim: Number of input features.
    - output_dim: Number of output features (1 for regression or number of classes for classification).
    - task_type: "regression" or "classification".

    Returns:
    - model: Compiled Keras MLP model.
    """
    model = Sequential()

    # Input Layer
    model.add(Dense(256, input_dim=input_dim, activation='relu'))
  
    # Hidden Layers
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(32, activation='relu'))

    # Output Layer
    if task_type == "regression":
        model.add(Dense(output_dim, activation='linear'))  # Linear activation for regression
        loss_function = 'mean_squared_error'  # Regression loss
    elif task_type == "classification":
        if output_dim == 1:
            model.add(Dense(output_dim, activation='sigmoid'))  # Sigmoid for binary classification
            loss_function = 'binary_crossentropy'  # Binary classification loss
        else:
            model.add(Dense(output_dim, activation='softmax'))  # Softmax for multi-class classification
            loss_function = 'sparse_categorical_crossentropy'  # Multi-class classification loss
    else:
        raise ValueError("task_type must be 'regression' or 'classification'.")

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0008),
                  loss=loss_function,
                  metrics=['accuracy'] if task_type == "classification" else ['mae'])

    return model

input_dim = len(features)  # Number of input features
output_dim = 1  # Single output for regression
task_type = "regression"  # Choose "regression" or "classification"

mlp_model = create_mlp(input_dim, output_dim, task_type)


mlp_model.summary()

# Train the model

mlp_model.fit(X_train, y_train_np, validation_data=(X_val, y_val_np), epochs=50, batch_size=64, verbose=1)


### 9-Wide and Deep NN

In [ ]:
from tensorflow.keras.layers import concatenate, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K
import keras_tuner as kt

def combined_loss(y_true, y_pred):
    rmse = K.sqrt(K.mean(K.square(y_true - y_pred)))
    mean_error = K.mean(y_true - y_pred)
    return rmse + 0.1 * K.abs(mean_error)  # Adjust weight for bias correction


# Wide Branch
input_wide = Input(shape=(X_train.shape[1],))
wide_output = Dense(32, activation='relu')(input_wide)

# Deep Branch
input_deep = Input(shape=(X_train.shape[1],))
x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(input_deep)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x) 
x = Dropout(0.2)(x)  
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x) 
x = Dropout(0.2)(x) 
deep_output = Dense(32, activation='relu')(x)

# Combine Wide and Deep
merged = concatenate([wide_output, deep_output])
output = Dense(1, activation='linear')(merged)

model_wide_deep = Model(inputs=[input_wide, input_deep], outputs=output)
model_wide_deep.compile(optimizer=Adam(learning_rate=0.0005), loss=combined_loss)

# Prepare the data
X_train_wide = X_train  
X_train_deep = X_train 
X_test_wide = X_test 
X_test_deep = X_test
X_val_wide = X_val
X_val_deep = X_val

# Train the model
history_wide_deep = model_wide_deep.fit(
    [X_train_wide, X_train_deep],  # Two inputs
    y_train_np,  # Single target
    validation_data=([X_val_wide, X_val_deep],y_val_np),
    epochs=100,
    batch_size=128,
    verbose=1
)

# Evaluate the model
val_loss = model_wide_deep.evaluate([X_val_wide, X_val_deep], y_val_np, verbose=1)
print(f"Val Loss: {val_loss:.4f}")


### 10-Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
#Random Forest algorithm variables are defined
rf_model = RandomForestRegressor(n_estimators=150, max_depth=20, random_state=42)
#Model Training
rf_model.fit(X_train, y_train_np)
# Evaluating on the validation set
y_val_pred = rf_model.predict(X_val)

mae = mean_absolute_error(y_val_np, y_val_pred)
rmse = np.sqrt(mean_squared_error(y_val_np, y_val_pred))

r2 = r2_score(y_val_np, y_val_pred)

print(f"Validation MAE: {mae:.4f}")
print(f"Validation RMSE: {rmse:.4f}")
print(f"Validation R² Score: {r2:.4f}")

### 11-KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

#KNN algorithm variables are defined
knn = KNeighborsRegressor(n_neighbors=7, metric='manhattan',weights='distance')

#Model Training
knn.fit(X_train, y_train_np)
# Evaluating on the validation set
y_pred = knn.predict(X_val)

mse = mean_squared_error(y_val_np, y_pred)
r2 = r2_score(y_val_np, y_pred)

print("mse",mse)
print("r2 score",r2)

### 12-Light GBM

In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

train_data = lgb.Dataset(X_train, label=y_train_np)
test_data = lgb.Dataset(X_test, label=y_test_np, reference=train_data)

# Light GBM variables are defined
params = {
    'objective': 'regression',  
    'metric': 'mse', 
    'boosting_type': 'gbdt', 
    'num_leaves': 64,  
    'learning_rate': 0.05,  
    'feature_fraction': 0.9,  
    'lambda_l2': 0.1 
}

# Model Training
num_round = 500
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# Evaluating on the validation set
y_pred = bst.predict(X_val, num_iteration=bst.best_iteration)


mse = mean_squared_error(y_val_np, y_pred)
print(f'Mean Squared Error: {mse}')

mae = mean_absolute_error(y_val_np, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

r2 = r2_score(y_val_np, y_pred)

print(f"Validation MAE: {mae:.4f}")
print(f"Validation RMSE: {rmse:.4f}")
print(f"Validation R² Score: {r2:.4f}")

### 13-XG Boost

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

dtrain = xgb.DMatrix(X_train, label=y_train_np)
dval = xgb.DMatrix(X_val, label=y_val_np)

# XG Boost variables are defined
params = {
    'objective': 'reg:squarederror', 
    'eval_metric': 'rmse',  
    'learning_rate': 0.05, 
    'max_depth': 6, 
    'colsample_bytree': 0.8,  
    'subsample': 0.8, 
    
}

# Model Training
xgb_reg = xgb.train(params, dtrain, num_boost_round=500, evals=[(dval, 'test')], early_stopping_rounds=20, verbose_eval=50)

# Evaluating on the validation set
y_pred = xgb_reg.predict(dval)

mae = mean_absolute_error(y_val_np, y_pred)
mse = mean_squared_error(y_val_np, y_pred)
rmse = np.sqrt(mean_squared_error(y_val_np, y_pred))
r2 = r2_score(y_val_np, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"rmse: {rmse:.4f}")

### 14-Decision Tree

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Decision Tree variables are defined
dt_regressor = DecisionTreeRegressor(max_depth=15, min_samples_split=10, min_samples_leaf=5,criterion='squared_error')

# Model Training
dt_regressor.fit(X_train, y_train_np)

# Evaluating on validation set
y_pred = dt_regressor.predict(X_val)


mse = mean_squared_error(y_val_np, y_pred)
r2 = r2_score(y_val_np, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# To save model history

save_model_and_history(model, history, "Fully_Connected")
save_model_and_history(modelBatch, historyBatch, "Batch_Normalization_NN")
save_model_and_history(model_leaky, historyLeaky, "Leaky_ReLU_NN")
save_model_and_history(model_residual, history_residual, "Residual_NN")
save_model_and_history(model_deep, history_deep, "Deep_NN")
save_model_and_history(model_attention, history_attention , "Transformer_Multihead_NN")
save_model_and_history(regression_model, history_encoder, "Autoencoder_Regression_NN")
save_model_and_history(model_wide_deep, history_wide_deep, "WideAndDeep_NN")

In [ ]:

# Function is defined to plot training history
def plot_training_history(history, model_name):
    """
    Plots training and validation loss for a given model.
    
    Parameters:
    - history: Keras History object returned by model.fit().
    - model_name: Name of the model to display in the plot title.
    """
    save_dir ="folder/Results/" # To give directory to save plot history
    os.makedirs(save_dir, exist_ok=True)
    plt.figure(figsize=(8, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {model_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.savefig(f"{save_dir}/{model_name.replace(' ', '_')}_loss.jpeg", dpi=300)
    plt.close()  



# Plot each model's training history
plot_training_history(history, "Fully Connected NN")
plot_training_history(historyBatch, "Batch Normalization NN")
plot_training_history(historyLeaky, "Leaky ReLU NN")
plot_training_history(history_residual, "Residual NN")
plot_training_history(history_deep, "Deep NN")
plot_training_history(mlp_model, "MLP")
plot_training_history(history_attention, "Multi-Head Attention NN")
plot_training_history(history_encoder, "Autoencoder Regression NN")
plot_training_history(history_wide_deep, "Wide and Deep NN")